# Carcrash Severity Prediction

Downloading the datasets:

Please run `wget https://data.wprdc.org/dataset/3130f583-9499-472b-bb5a-f63a6ff6059a/resource/ec578660-2d3f-489d-9ba1-af0ebfc3b140/download/all-crashes-2004-2018.csv.zip`
or visit this link to get the Allegheny County crash data.

## Load Data

In [17]:
import ckanapi
from pprint import pprint
import pandas as pd
from sklearn.model_selection import train_test_split


site = "https://data.wprdc.org"

In [5]:
def get_resource_data(site,resource_id,count=50):
    # Use the datastore_search API endpoint to get <count> records from
    # a CKAN resource.
    ckan = ckanapi.RemoteCKAN(site)
    response = ckan.action.datastore_search(id=resource_id, limit=count)

    # A typical response is a dictionary like this
    #{u'_links': {u'next': u'/api/action/datastore_search?offset=3',
    #             u'start': u'/api/action/datastore_search'},
    # u'fields': [{u'id': u'_id', u'type': u'int4'},
    #             {u'id': u'pin', u'type': u'text'},
    #             {u'id': u'number', u'type': u'int4'},
    #             {u'id': u'total_amount', u'type': u'float8'}],
    # u'limit': 3,
    # u'records': [{u'_id': 1,
    #               u'number': 11,
    #               u'pin': u'0001B00010000000',
    #               u'total_amount': 13585.47},
    #              {u'_id': 2,
    #               u'number': 2,
    #               u'pin': u'0001C00058000000',
    #               u'total_amount': 7827.64},
    #              {u'_id': 3,
    #               u'number': 1,
    #               u'pin': u'0001C01661006700',
    #               u'total_amount': 3233.59}],
    # u'resource_id': u'd1e80180-5b2e-4dab-8ec3-be621628649e',
    # u'total': 88232}
    data = response['records']
    return data

In [6]:
crash_data_2017 = get_resource_data(site,resource_id="bf8b3c7e-8d60-40df-9134-21606a451c1a",count=999999999) 

In [7]:
len(crash_data_2017)
df = pd.DataFrame(crash_data_2017)
print(df.head())

  ACCESS_CTRL ADJ_RDWY_SEQ  AGGRESSIVE_DRIVING  ALCOHOL_RELATED  \
0        None         None                   0                0   
1        None         None                   1                0   
2        None         None                   0                0   
3        None         None                   0                0   
4        None         None                   0                1   

   AUTOMOBILE_COUNT  BELTED_DEATH_COUNT  BELTED_MAJ_INJ_COUNT  BICYCLE  \
0                 2                   0                     0        0   
1                 1                   0                     0        0   
2                 1                   0                     0        0   
3                 1                   0                     0        0   
4                 4                   0                     0        0   

   BICYCLE_COUNT  BICYCLE_DEATH_COUNT  ...  WORK_ZONE_LOC  WORK_ZONE_TYPE  \
0              0                    0  ...           None            None  

## Data Visualization

TODO: Add a graph/animation to show the car crash data
maybe the geospatial data?

## Data Cleanup

In [27]:
import pandas as pd
import wget
from pathlib import Path
import numpy as np

#clean sch_bus_ind y/n to integer 0,1

def type_boolean(c):
    if c == "Y": return 1
    elif c == "N": return 0
    # elif c == "nan": return np.nan
    else:
        return np.nan
    # raise ValueError(c)

def ROAD_CONDITION(c): # 8 is other 9 is unknown, 1,7->2, 3->4, 4->3, 5,6->5, 2,8,9->nan
    if c == 1 or c == 7:
        return 2
    elif c == 3:
        return 4
    elif c == 4:
        return 3
    elif c == 5 or c == 6:
        return 5
    else:
        return np.nan

def INTERSECT_TYPE(c): # 10 is other 99 is unkonw
    if c <= 9:
        return c
    else:
        return np.nan

def ILLUMINATION(c):
    if c <= 6:
        return c
    else:
        return np.nan
    
def WEATHER(c):
    if c <= 7:
        return c
    else:
        return np.nan
    
def TIME(c): # extract only the hour
    if c:
        return c // 100
    else:
        return np.nan

if not Path('all-crashes-2004-2018.csv.zip').exists():
    wget.download("https://data.wprdc.org/dataset/3130f583-9499-472b-bb5a-f63a6ff6059a/resource/ec578660-2d3f-489d-9ba1-af0ebfc3b140/download/all-crashes-2004-2018.csv.zip")
# zf = zipfile.ZipFile('all-crashes-2004-2018.csv.zip') 
df_io = pd.read_csv('all-crashes-2004-2018.csv.zip')
# print(df.head())
# print(list(df))
# static_columns = "ROAD_CONDITION,INTERSECT_TYPE,URBAN_RURAL,DISTRICT,STATE_ROAD,LOCAL_ROAD,SNOW_SLUSH_ROAD,LANE_CLOSED,TIME_OF_DAY,SPEED_LIMIT"
# dynamic_columns = "ILLUMINATION,MOTORCYCLE_COUNT,HEAVY_TRUCK_COUNT,WEATHER,HAZARDOUS_TRUCK,SCH_BUS_IND,AUTOMOBILE_COUNT"
# output_columns = "PERSON_COUNT,FATAL_COUNT,INJURY_COUNT,MAX_SEVERITY_LEVEL,MAJOR_INJURY"
# df_io = df[(static_columns+","+dynamic_columns+","+output_columns).split(',')]
# print(df_io.head())
# print(df_io.dtypes)
# print(df_io.info())

# df_io['SCH_BUS_IND'] = df_io['SCH_BUS_IND'].apply(type_boolean)



# df_io['ROAD_CONDITION'] = df_io['ROAD_CONDITION'].apply(ROAD_CONDITION)
# df_io['INTERSECT_TYPE'] = df_io['INTERSECT_TYPE'].apply(INTERSECT_TYPE)
# df_io['ILLUMINATION'] = df_io['ILLUMINATION'].apply(ILLUMINATION)
# df_io['WEATHER'] = df_io['WEATHER'].apply(WEATHER)
# df_io['TIME_OF_DAY'] = df_io['TIME_OF_DAY'].apply(TIME)

# df_io = df_io.astype("Int64")

# print(df_io.head())
# print(df_io.dtypes)
# print(df_io.info())

# drop col that will not be used
static = ['ROAD_CONDITION', 'INTERSECT_TYPE', 'DISTRICT', 'LANE_CLOSED', 'TIME_OF_DAY', 'SPEED_LIMIT', 'ILLUMINATION']
dynamic = ['MOTORCYCLE_COUNT', 'HEAVY_TRUCK_COUNT', 'HAZARDOUS_TRUCK', 'AUTOMOBILE_COUNT', 'SCH_BUS_IND', 'WEATHER']
label = ['PERSON_COUNT', 'FATAL_COUNT', 'INJURY_COUNT', 'MAX_SEVERITY_LEVEL', 'MAJOR_INJURY']
data = static + dynamic
for col in df_io.columns:
    if col not in static and col not in dynamic and col not in label:
        df_io.drop(col, axis = 1, inplace = True)  
# clean data
df_io['SCH_BUS_IND'] = df_io['SCH_BUS_IND'].apply(type_boolean)
df_io['ROAD_CONDITION'] = df_io['ROAD_CONDITION'].apply(ROAD_CONDITION)
df_io['INTERSECT_TYPE'] = df_io['INTERSECT_TYPE'].apply(INTERSECT_TYPE)
df_io['ILLUMINATION'] = df_io['ILLUMINATION'].apply(ILLUMINATION)
df_io['WEATHER'] = df_io['WEATHER'].apply(WEATHER)
df_io['TIME_OF_DAY'] = df_io['TIME_OF_DAY'].apply(TIME)

df_io = df_io.astype("Int64")
# drop rows contain nan
df_io = df_io.dropna()

print(df_io[:5])
# group data into dataset/label
df_data = df_io[data].copy()
df_label = df_io[label].copy()
print(df_data.info())
print(df_label.info())

   DISTRICT  TIME_OF_DAY  ILLUMINATION  WEATHER  ROAD_CONDITION  \
1        11           10             1        1               5   
3        11            9             1        1               2   
5        11           14             1        1               2   
8        11           12             1        2               2   
9        11           11             1        2               2   

   INTERSECT_TYPE  SCH_BUS_IND  PERSON_COUNT  AUTOMOBILE_COUNT  \
1               0            0             1                 1   
3               1            0             3                 1   
5               1            0             2                 2   
8               0            0             2                 2   
9               1            0             2                 2   

   MOTORCYCLE_COUNT  HEAVY_TRUCK_COUNT  FATAL_COUNT  INJURY_COUNT  \
1                 0                  0            0             1   
3                 0                  1            0           

## Train Test Split

In [29]:
# split the dataset with 33% as train data
X_train, X_test, y_train, y_test = train_test_split(df_data, df_label, test_size=0.33, random_state=42)
print(X_train.info())
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29737 entries, 127006 to 60503
Data columns (total 13 columns):
ROAD_CONDITION       29737 non-null Int64
INTERSECT_TYPE       29737 non-null Int64
DISTRICT             29737 non-null Int64
LANE_CLOSED          29737 non-null Int64
TIME_OF_DAY          29737 non-null Int64
SPEED_LIMIT          29737 non-null Int64
ILLUMINATION         29737 non-null Int64
MOTORCYCLE_COUNT     29737 non-null Int64
HEAVY_TRUCK_COUNT    29737 non-null Int64
HAZARDOUS_TRUCK      29737 non-null Int64
AUTOMOBILE_COUNT     29737 non-null Int64
SCH_BUS_IND          29737 non-null Int64
WEATHER              29737 non-null Int64
dtypes: Int64(13)
memory usage: 3.5 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 14648 entries, 60038 to 36144
Data columns (total 13 columns):
ROAD_CONDITION       14648 non-null Int64
INTERSECT_TYPE       14648 non-null Int64
DISTRICT             14648 non-null Int64
LANE_CLOSED          14648 non-null Int64
TIME_OF_DAY    

## Environment Static Analysis


## Dynamic Analysis

## Combined Analysis

## Real-Time Severity Prediction